**We Use BERT MODEL to Classify the user comments about the application. The Model Classify the comments into Application Fuctional Problems or Application Content Problems where the team can use this to find the app problems and solve them efficiently.**

Import Libraries.

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Load your dataset
data = pd.read_csv('CommentsRealDataset.csv', names=['comment', 'label'])

# Split the data into training and testing sets
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)


In [ ]:
data.head()


,comment,label
0,This app is cloned,0
1,to political,1
2,This app is overvalued,1
3,Were the hell is Jay Leno's show?,1
4,I have reinstalled the app numerous times beca...,0


In [ ]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

def tokenize_data(data):
    return tokenizer(
        data['comment'].tolist(),
        padding=True,
        truncation=True,
        return_tensors='tf'
    )

train_tokens = tokenize_data(train_data)
test_tokens = tokenize_data(test_data)


In [ ]:
print(train_data['label'].head())
print(test_data['label'].head())


371     0
306     0
1096    0
10      1
535     1
Name: label, dtype: int64
597     1
715     0
921     0
1159    1
415     1
Name: label, dtype: int64


In [ ]:
train_labels = tf.convert_to_tensor(train_data['label'].values, dtype=tf.int32)
test_labels = tf.convert_to_tensor(test_data['label'].values, dtype=tf.int32)


In [ ]:
train_dataset = tf.data.Dataset.from_tensor_slices((dict(train_tokens), train_labels)).batch(32)
test_dataset = tf.data.Dataset.from_tensor_slices((dict(test_tokens), test_labels)).batch(32)


In [ ]:
from transformers import TFBertForSequenceClassification
import tensorflow as tf

model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

# Compile the model with a specific loss function
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=5e-5),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])



All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
print(train_labels)
print(test_labels)
print(train_tokens)
print(test_tokens)


In [ ]:
history = model.fit(
    train_dataset,
    validation_data=test_dataset,
    epochs=3
)


Epoch 1/3
31/31 [==============================] - 1877s 59s/step - loss: 0.5032 - accuracy: 0.7602 - val_loss: 0.3749 - val_accuracy: 0.8327
Epoch 2/3
31/31 [==============================] - 1758s 57s/step - loss: 0.2423 - accuracy: 0.9000 - val_loss: 0.3605 - val_accuracy: 0.8449
Epoch 3/3
31/31 [==============================] - 1759s 57s/step - loss: 0.1168 - accuracy: 0.9541 - val_loss: 0.5482 - val_accuracy: 0.8612


In [ ]:
# Save the model
model.save('my_model')




In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Save the model
model.save_weights('/content/drive/MyDrive/CommentClassification_weights.h5')






Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
from transformers import TFBertForSequenceClassification
import tensorflow as tf
# Load the model architecture first
model2 = TFBertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

# Load the weights
model2.load_weights('/content/drive/MyDrive/CommentClassification_weights.h5')


All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from transformers import BertTokenizer, TFBertForSequenceClassification
import tensorflow as tf

# Load the tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
# model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

# Tokenize the new comment
new_comment = "Whenever I open this app in my samsung galaxy mobile it alwasy crashes"
new_tokens = tokenizer(
    new_comment,
    padding=True,
    truncation=True,
    return_tensors='tf'
)
# [0.6,0.4]

# Make the prediction
predictions = model2.predict(new_tokens['input_ids'])
predicted_label = tf.argmax(predictions.logits, axis=1).numpy()

# Interpret the prediction
if predicted_label == 0:
    print("Developer")
else:
    print("Business")


1/1 [==============================] - 6s 6s/step
Developer


In [ ]:
comments = ["why there is no dark mode", "app crashes everytime", "cannot find Indian stocks", "slow and cloned"]
inputs = tokenizer(comments, padding=True, truncation=True, return_tensors='tf')

predictions = model2.predict(inputs['input_ids'])
predicted_labels = tf.argmax(predictions.logits, axis=1).numpy()

print(predicted_labels)

for i in range(len(predicted_labels)):
  if(predicted_labels[i]==0):
    print(f"{comments[i] } - Developer")

  else:
    print(f"{comments[i] } - Business")

1/1 [==============================] - 0s 191ms/step
[1 0 1 0]
why there is no dark mode - Business
app crashes everytime - Developer
cannot find Indian stocks - Business
slow and cloned - Developer


In [ ]:
# Tokenize the test data
test_tokens = tokenizer(
    test_data['comment'].tolist(),
    padding=True,
    truncation=True,
    return_tensors='tf'
)

# Make predictions
predictions = model2.predict(test_tokens['input_ids'])
predicted_labels = tf.argmax(predictions.logits, axis=1).numpy()

# Evaluate the model
from sklearn.metrics import accuracy_score, classification_report

true_labels = test_data['label'].values
accuracy = accuracy_score(true_labels, predicted_labels)
report = classification_report(true_labels, predicted_labels, target_names=['Developer', 'Business'])

print(f'Accuracy: {accuracy}')
print('Classification Report:')
print(report)
